## Imports and Setup

In [1]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('/home/itingzon.unicef/giga')
from src import sat_download
from utils import post_utils
from utils import calib_utils
from utils import config_utils
from utils import data_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [10]:
iso_code="SEN"
sum_threshold=0
buffer_size=50
calibrator="LogisticCalibration"

data_config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AF.yaml")
model_config_file = os.path.join(os.getcwd(), "configs/cnn_configs/convnext_large_v01.yaml")
data_config = config_utils.load_config(data_config_file)
model_config = config_utils.load_config(model_config_file)

## Load Model Predictions

In [11]:
preds = post_utils.load_preds(
    iso_code, data_config, model_config, sum_threshold, buffer_size, calibrator
)
preds.head(2)

Reading files for SEN...: 100%|████████████████████| 45/45 [00:03<00:00, 14.20it/s]                                     
INFO:root:Data dimensions: (14021, 10)


,prob,UID,sum,group,geometry,ADM1,ADM2,ADM3,PUID,prob_cal
8484,0.982964,44707,38.0,7282,POINT (-1766669.174 1444139.279),Sedhiou,Sedhiou,Djibabouya,Sedhiou_44707,0.983596
11449,0.982403,131374,28.0,9804,POINT (-1686689.864 1574843.369),Kaffrine,Malem Hodar,Sagna,Malem Hodar_131374,0.982973


## Load Government Data

In [12]:
master = post_utils.load_master(iso_code, data_config)
master.head(2)

INFO:root:clean
0    9033
2     794
1     238
Name: count, dtype: int64
INFO:root:Data dimensions: (10065, 13)


,MUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,UNICEF-SEN-SCHOOL-00000000,UNICEF,SEN,Senegal,Africa,Sub-Saharan Africa,EE ALIEU SAMB,7614c6c6-9aca-36ff-978b-22bfca59916a,0,POINT (-17.51261 14.74680),Dakar,Dakar,Almadies
1,UNICEF-SEN-SCHOOL-00000001,UNICEF,SEN,Senegal,Africa,Sub-Saharan Africa,EE EL HADJI BAYTIR SAMB,350fb172-bfef-331d-bd55-5c51139b9f48,0,POINT (-17.51196 14.74951),Dakar,Dakar,Almadies


## Load OSM and Overture Data

In [13]:
osm_overture = post_utils.load_osm_overture(iso_code, data_config)
osm_overture.head(2)

INFO:root:clean
0    784
Name: count, dtype: int64
INFO:root:Data dimensions: (784, 13)


,SUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,OSM-SEN-SCHOOL-00000000,OSM,SEN,Senegal,Africa,Sub-Saharan Africa,Ecole Nationale de Police,None,0,POINT (-17.44890 14.68903),Dakar,Dakar,Plateau
1,OSM-SEN-SCHOOL-00000001,OSM,SEN,Senegal,Africa,Sub-Saharan Africa,CIFOP,None,0,POINT (-16.90105 15.16665),Thies,Tivaoune,Meouane


## Calculate Distances

In [14]:
preds, master, osm_overture = post_utils.calculate_nearest_distances(preds, master, osm_overture)
preds.head(2)

,prob,UID,sum,group,geometry,ADM1,ADM2,ADM3,PUID,prob_cal,distance_to_nearest_master,MUID,distance_to_nearest_osm_overture,SUID
8484,0.982964,44707,38.0,7282,POINT (-15.87026 12.86348),Sedhiou,Sedhiou,Djibabouya,Sedhiou_44707,0.983596,31.647240,UNICEF-SEN-SCHOOL-00001558,8003.396762,OSM-SEN-SCHOOL-00000457
11449,0.982403,131374,28.0,9804,POINT (-15.15179 14.00546),Kaffrine,Malem Hodar,Sagna,Malem Hodar_131374,0.982973,28.146039,UNICEF-SEN-SCHOOL-00003603,17292.290044,OSM-SEN-SCHOOL-00000456


## Save Results

In [15]:
post_utils.save_results(iso_code, preds, model_config, source="preds")
post_utils.save_results(iso_code, master, model_config, source="master")
post_utils.save_results(iso_code, osm_overture, model_config, source="osm_overture")